# Correcting the map counts to account for diving seals and higher on-the-ground counts
This file includes the code that inflates the map counts using detection rate-estimating functions fitted with data from Erebus Bay, where ground counts could be conducted.  
  
In a nutshell, we use these models to estimate the detection rate, and then inflate the map counts according to this rate.


In [17]:
## Clear memory
rm(list=ls())
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,679440,36.3,1168576,62.5,1168576,62.5
Vcells,1162638,8.9,5246909,40.1,8198173,62.6


### Loading libraries, functions and data

In [18]:
libs<-c("ggplot2","plyr","dplyr")
lapply(libs, require, character.only = TRUE)
pathToLocalGit<-"/home/ubuntu/Workspace/ContinentalWESEestimates/"

source(paste0(pathToLocalGit,"scripts/countSealsFromTags_functions.R"))

## Load the data - the data were generated from notebook: "Count Seals From Tags.ipynb"
load(file=paste0(pathToLocalGit,"data/estimatesByMap_unadjusted.RData"))
head(df)

## Print and check that this is the correct version of the data
estByRegionQ<-as.data.frame(mapcounts[,c("region","lowerH","estimateH","upperH")] %>% group_by(region) %>% dplyr::summarize(lower=ceiling(sum(lowerH)),estimate=ceiling(sum(estimateH)),upper=ceiling(sum(upperH))))
estByRegionQ<-rbind(estByRegionQ,data.frame(region="Total",lower=round(sum(mapcounts$lowerH)),estimate=round(sum(mapcounts$estimateH)),upper=round(sum(mapcounts$upperH))))
print(estByRegionQ)

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

,regionMapId,originHour,lclNumSeals,estNumSeals,uclNumSeals,mapcoords.x1,mapcoords.x2,acquisition_date,region,satId,⋯,originTZ,year,totalTags,scaledTotalTags,numHour,sinH,resid,estimateH,lowerH,upperH
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,AMU101663,21,1,1,4,-85.48336,-73.34752,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,1,-0.33940735,0.1666667,0.8660254,-0.58817634,1,1,4
2,AMU102072,21,1,1,4,-85.44474,-73.41747,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,2,-0.29654759,0.1666667,0.8660254,-0.01598454,1,1,4
3,AMU102099,21,1,1,4,-85.79235,-73.17324,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,2,-0.29654759,0.1666667,0.8660254,-0.77273819,1,1,4
4,AMU103615,21,1,1,4,-85.79235,-73.17696,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,1,-0.33940735,0.1666667,0.8660254,-0.58817634,1,1,4
5,AMU103695,21,1,1,4,-85.79235,-73.16951,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,1,-0.33940735,0.1666667,0.8660254,-0.58781338,1,1,4
6,AMU103791,21,1,6,14,-85.81166,-73.17324,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,8,-0.03938903,0.1666667,0.8660254,3.12751275,6,1,14


  region lower estimate  upper
1    AMU  1013     2189   6223
2    EA1  2375     5967  16126
3    EA2  2174     5063  13934
4    QMA  2833     7142  19337
5    RSS  8330    22333  59682
6    WAP  3892     8894  22417
7  Total 20617    51588 137719


Now we use the handy function to request the detection rates under both models. Note that we can specify the "weight" of island counts. This is the proportion of map locations that resemble the islet haul-out locations in Erebus Bay (i.e., Turk's Head-Tryggve and Hutton Cliffs). According to Michelle and David, the vast majority of counts came from such locations. Here we are conservative and assume 95% of locations are islets.

In [19]:
names(mapcounts)<-gsub("scaledTotalTags","scaledNumTags",names(mapcounts))

adjRates<-predictDetRates(pathToGit=pathToLocalGit,dat=mapcounts,keyFieldName="regionMapId",islandWeight=0.95)
head(adjRates)

,regionMapId,wgtPredColRate,wgtPredIslRate,Year
,<chr>,<dbl>,<dbl>,<chr>
1,AMU101663,0.2975275,0.2406196,2010
2,AMU102072,0.3012151,0.2432923,2010
3,AMU102099,0.3012151,0.2432923,2010
4,AMU103615,0.2975275,0.2406196,2010
5,AMU103695,0.2975275,0.2406196,2010
6,AMU103791,0.3233407,0.2593285,2010


Finally, we add the detection rates to our data.frame of map counts, inflate, and summarize the results.  
  
Since detectionRate = count-in-map/count-on-the-ground, and we want count-on-the-ground, then:
Count-on-the-ground = count-in-map/detectionRate

In [34]:
countdf<-merge(mapcounts,adjRates,by="regionMapId")
nrow(countdf)==2*nrow(mapcounts)

countdf$mdlColEstimate<-round(countdf$estimateH/countdf$wgtPredColRate)
countdf$mdlColUpper<-round(countdf$upperH/countdf$wgtPredColRate)
countdf$mdlColLower<-round(countdf$lowerH/countdf$wgtPredColRate)

countdf$mdlIslEstimate<-round(countdf$estimateH/countdf$wgtPredIslRate)
countdf$mdlIslUpper<-round(countdf$upperH/countdf$wgtPredIslRate)
countdf$mdlIslLower<-round(countdf$lowerH/countdf$wgtPredIslRate)

## Using the colony model for 2010:
print("Colony model with reference year 2010")
countdf10<-subset(countdf,Year=="2010")
estByRegionCol<-as.data.frame(countdf10[,c("region","mdlColLower","mdlColEstimate","mdlColUpper")] %>% group_by(region) %>% dplyr::summarize(Lower=round(sum(mdlColLower)),Estimate=round(sum(mdlColEstimate)),Upper=round(sum(mdlColUpper))))
estByRegionCol<-rbind(estByRegionCol,data.frame(region="Total",Lower=round(sum(countdf10$mdlColLower)),Estimate=round(sum(countdf10$mdlColEstimate)),Upper=round(sum(countdf10$mdlColUpper))))
print(estByRegionCol)
cat("\n")
## Using the colony model for 2011:
print("Colony model with reference year 2011")
countdf11<-subset(countdf,Year=="2011")
estByRegionCol<-as.data.frame(countdf11[,c("region","mdlColLower","mdlColEstimate","mdlColUpper")] %>% group_by(region) %>% dplyr::summarize(Lower=round(sum(mdlColLower)),Estimate=round(sum(mdlColEstimate)),Upper=round(sum(mdlColUpper))))
estByRegionCol<-rbind(estByRegionCol,data.frame(region="Total",Lower=round(sum(countdf11$mdlColLower)),Estimate=round(sum(countdf11$mdlColEstimate)),Upper=round(sum(countdf11$mdlColUpper))))
print(estByRegionCol)

[1] TRUE

[1] "Colony model with reference year 2010"
  region Lower Estimate  Upper
1    AMU  2979     6426  18879
2    EA1  6829    16694  46660
3    EA2  6276    14198  40576
4    QMA  8057    19420  54635
5    RSS 23104    58132 161958
6    WAP  9289    22982  62684
7  Total 56534   137852 385392

[1] "Colony model with reference year 2011"
  region Lower Estimate  Upper
1    AMU  3907     8004  23639
2    EA1  8868    20540  57886
3    EA2  8172    17600  50422
4    QMA 10429    23899  67525
5    RSS 29753    71088 199298
6    WAP 12044    28352  77568
7  Total 73173   169483 476338


In [35]:
## Using the islet/mainland model for 2010:
print("Islet/mainland model with reference year 2010")
countdf10<-subset(countdf,Year=="2010")
estByRegionIsl<-as.data.frame(countdf10[,c("region","mdlIslLower","mdlIslEstimate","mdlIslUpper")] %>% group_by(region) %>% dplyr::summarize(Lower=round(sum(mdlIslLower)),Estimate=round(sum(mdlIslEstimate)),Upper=round(sum(mdlIslUpper))))
estByRegionIsl<-rbind(estByRegionIsl,data.frame(region="Total",Lower=round(sum(countdf10$mdlIslLower)),Estimate=round(sum(countdf10$mdlIslEstimate)),Upper=round(sum(countdf10$mdlIslUpper))))
print(estByRegionIsl)
cat("\n")
## Using the islet/mainland model for 2011:
print("Islet/mainland model with reference year 2011")
countdf11<-subset(countdf,Year=="2011")
estByRegionIsl<-as.data.frame(countdf11[,c("region","mdlIslLower","mdlIslEstimate","mdlIslUpper")] %>% group_by(region) %>% dplyr::summarize(Lower=round(sum(mdlIslLower)),Estimate=round(sum(mdlIslEstimate)),Upper=round(sum(mdlIslUpper))))
estByRegionIsl<-rbind(estByRegionIsl,data.frame(region="Total",Lower=round(sum(countdf11$mdlIslLower)),Estimate=round(sum(countdf11$mdlIslEstimate)),Upper=round(sum(countdf11$mdlIslUpper))))
print(estByRegionIsl)

[1] "Islet/mainland model with reference year 2010"
  region Lower Estimate  Upper
1    AMU  3937     8130  23727
2    EA1  8984    21141  58764
3    EA2  8292    18035  51080
4    QMA 10582    24689  68832
5    RSS 30374    73950 204640
6    WAP 12217    29190  78964
7  Total 74386   175135 486007

[1] "Islet/mainland model with reference year 2011"
  region Lower Estimate  Upper
1    AMU  4865     9874  28553
2    EA1 11057    25468  70315
3    EA2 10167    21785  61203
4    QMA 12992    29633  82237
5    RSS 37122    88397 243273
6    WAP 15014    35122  94453
7  Total 91217   210279 580034


In [37]:
head(countdf); names(countdf)

,regionMapId,originHour,region,satId,scaledNumTags,numViews,numTaggers,resid,estNumSeals,lclNumSeals,⋯,upperH,wgtPredColRate,wgtPredIslRate,Year,mdlColEstimate,mdlColUpper,mdlColLower,mdlIslEstimate,mdlIslUpper,mdlIslLower
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,AMU101663,21,AMU,WV01,-0.3394073,8,1,-0.58817634,1,1,⋯,4,0.2367743,0.1963025,2011,4,17,4,5,20,5
2,AMU101663,21,AMU,WV01,-0.3394073,8,1,-0.58817634,1,1,⋯,4,0.2975275,0.2406196,2010,3,13,3,4,17,4
3,AMU102072,21,AMU,WV01,-0.2965476,6,2,-0.01598454,1,1,⋯,4,0.3012151,0.2432923,2010,3,13,3,4,16,4
4,AMU102072,21,AMU,WV01,-0.2965476,6,2,-0.01598454,1,1,⋯,4,0.2404619,0.1989752,2011,4,17,4,5,20,5
5,AMU102099,21,AMU,WV01,-0.2965476,6,1,-0.77273819,1,1,⋯,4,0.2404619,0.1989752,2011,4,17,4,5,20,5
6,AMU102099,21,AMU,WV01,-0.2965476,6,1,-0.77273819,1,1,⋯,4,0.3012151,0.2432923,2010,3,13,3,4,16,4


[1] "regionMapId"    "originHour"     "region"         "satId"         
 [5] "scaledNumTags"  "numViews"       "numTaggers"     "resid"         
 [9] "estNumSeals"    "lclNumSeals"    "uclNumSeals"    "mapcoords.x1"  
[13] "mapcoords.x2"   "propL"          "propU"          "sinH"          
[17] "predictH"       "estimateH"      "lowerH"         "upperH"        
[21] "wgtPredColRate" "wgtPredIslRate" "Year"           "mdlColEstimate"
[25] "mdlColUpper"    "mdlColLower"    "mdlIslEstimate" "mdlIslUpper"   
[29] "mdlIslLower"

In [38]:
save(countdf, file=paste0(pathToLocalGit,"data/FinalWESEcounts.RData"))